# Notebook 3: KPI Calculation and Insights

**Objective:** Calculate metric and non-metric KPIs for stock performance using the cleaned SQLite table, with business insights.

---

## 1. Import Libraries
```python
import pandas as pd
import sqlite3
import numpy as np
from functools import reduce


In [ ]:
# Import required libraries
import pandas as pd
import sqlite3
import numpy as np
from functools import reduce

In [4]:
# Connect to SQLite database
conn = sqlite3.connect("stock_data.db")

# Load the cleaned stock data
df = pd.read_sql("SELECT * FROM google_stock", conn)

# Preview data
df.head()


,symbol,date,close,high,low,open,volume
0,GOOG,2016-06-14,718.27,722.47,713.1200,716.48,1306065
1,GOOG,2016-06-15,718.92,722.98,717.3100,719.00,1214517
2,GOOG,2016-06-16,710.36,716.65,703.2600,714.91,1982471
3,GOOG,2016-06-17,691.72,708.82,688.4515,708.65,3402357
4,GOOG,2016-06-20,693.71,702.48,693.4100,698.77,2082538


## KPI Metrics

In [14]:
# Average closing price
avg_close = df.groupby('symbol')['close'].mean().reset_index()
avg_close.rename(columns={'close': 'avg_close'}, inplace=True)
avg_close['insight'] = avg_close['avg_close'].apply(lambda x: f"Stock trades around ${x:.2f} on average.")
avg_close


,symbol,avg_close,insight
0,GOOG,1216.317067,Stock trades around $1216.32 on average.


In [16]:
# Price Volatility
volatility = df.groupby('symbol')['close'].std().reset_index()
volatility.rename(columns={'close': 'price_volatility'}, inplace=True)
volatility['insight'] = volatility['price_volatility'].apply(
    lambda x: "High volatility – stock price swings a lot." if x > 5 else "Low volatility – price relatively stable.")
volatility


,symbol,price_volatility,insight
0,GOOG,383.333358,High volatility – stock price swings a lot.


In [17]:
# Average Daily Range
df['daily_range'] = df['high'] - df['low']
avg_daily_range = df.groupby('symbol')['daily_range'].mean().reset_index()
avg_daily_range.rename(columns={'daily_range': 'avg_daily_range'}, inplace=True)
avg_daily_range['insight'] = avg_daily_range['avg_daily_range'].apply(
    lambda x: f"Typical intraday price movement: ${x:.2f}.")
avg_daily_range



,symbol,avg_daily_range,insight
0,GOOG,23.254504,Typical intraday price movement: $23.25.


In [18]:
# Average Trading Volume 
avg_volume = df.groupby('symbol')['volume'].mean().reset_index()
avg_volume.rename(columns={'volume': 'avg_volume'}, inplace=True)
avg_volume['insight'] = avg_volume['avg_volume'].apply(
    lambda x: "High liquidity" if x > 1_000_000 else "Low liquidity")
avg_volume


,symbol,avg_volume,insight
0,GOOG,1.601590e+06,High liquidity


## Non-Metric KPIs

In [19]:
# Upward Trend Periods
def upward_trend(group):
    return ((group['close'] > group['close'].shift(1)).sum())

upward_trends = df.groupby('symbol').apply(upward_trend).reset_index(name='upward_trend_days')
upward_trends['insight'] = upward_trends['upward_trend_days'].apply(
    lambda x: f"{x} days of upward movement observed.")
upward_trends


C:\Users\ACER\AppData\Local\Temp\ipykernel_6384\3797437508.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  upward_trends = df.groupby('symbol').apply(upward_trend).reset_index(name='upward_trend_days')


,symbol,upward_trend_days,insight
0,GOOG,695,695 days of upward movement observed.


In [20]:
# Downward Trend Periods
def downward_trend(group):
    return ((group['close'] < group['close'].shift(1)).sum())

downward_trends = df.groupby('symbol').apply(downward_trend).reset_index(name='downward_trend_days')
downward_trends['insight'] = downward_trends['downward_trend_days'].apply(
    lambda x: f"{x} days of downward movement observed.")
downward_trends


C:\Users\ACER\AppData\Local\Temp\ipykernel_6384\825302138.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  downward_trends = df.groupby('symbol').apply(downward_trend).reset_index(name='downward_trend_days')


,symbol,downward_trend_days,insight
0,GOOG,562,562 days of downward movement observed.


In [21]:
# High-Volume Spikes
def high_volume_spikes(group):
    threshold = group['volume'].mean() * 2
    return (group['volume'] > threshold).sum()

volume_spikes = df.groupby('symbol').apply(high_volume_spikes).reset_index(name='high_volume_spikes')
volume_spikes['insight'] = volume_spikes['high_volume_spikes'].apply(
    lambda x: f"{x} days with unusually high trading activity.")
volume_spikes


C:\Users\ACER\AppData\Local\Temp\ipykernel_6384\3599706775.py:6: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  volume_spikes = df.groupby('symbol').apply(high_volume_spikes).reset_index(name='high_volume_spikes')


,symbol,high_volume_spikes,insight
0,GOOG,54,54 days with unusually high trading activity.


In [22]:
# Gap Days
def gap_days(group):
    return ((abs(group['open'] - group['close'].shift(1)) / group['close'].shift(1)) > 0.01).sum()

gaps = df.groupby('symbol').apply(gap_days).reset_index(name='gap_days')
gaps['insight'] = gaps['gap_days'].apply(
    lambda x: f"{x} days with significant opening price gaps (>1%).")
gaps


C:\Users\ACER\AppData\Local\Temp\ipykernel_6384\2203193855.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gaps = df.groupby('symbol').apply(gap_days).reset_index(name='gap_days')


,symbol,gap_days,insight
0,GOOG,197,197 days with significant opening price gaps (...
